#### Test surface projection
    - Surfaces taken from https://github.com/cirmuw/EvolutionOfCorticalShape
    - only provides outer/pial surface
    - creation of inner surface follows https://github.com/cirmuw/EvolutionOfCorticalShape/blob/main/_mapping_tutorial/map_mouse_aba_to_civm_to_cprf.sh


In [1]:
from pathlib import Path

mouse_aba_dir = Path("/opt/animalfmritools/animalfmritools/data_template/MouseABA")
mouse_aba_surfaces = mouse_aba_dir / "surfaces"

!ls {mouse_aba_surfaces}

README.txt
sub-043ABA_species-Mus+musculus_hemi-L.inner.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.outer.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.sphere
sub-043ABA_species-Mus+musculus_hemi-L.sphere.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.surf
sub-043ABA_species-Mus+musculus_hemi-L.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere.reg
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere.reg.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.surf
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.inner.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.outer.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.sphere
sub-043ABA_species-Mus+musculus_hemi-R.sphere.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.surf
sub-043ABA_species-Mus+mu

In [2]:
# Pial surface
L_surface = mouse_aba_surfaces / "sub-043ABA_species-Mus+musculus_hemi-L.surf.gii"
R_surface = mouse_aba_surfaces / "sub-043ABA_species-Mus+musculus_hemi-R.surf.gii"

offset = 0.5

for hemi, surface in zip(["L", "R"], [L_surface, R_surface]):

    # Generate a pseudo cortical ribbon around the provided pial surface
    surface_metric = str(surface).replace("surf.gii", "func.gii")
    !wb_command -surface-coordinates-to-metric {surface} {surface_metric} # convert to metric file

    surface_norm = str(surface).replace("surf.gii", "normals.func.gii")
    !wb_command -surface-normals {surface} {surface_norm} # generate surface normals

    # generate inner ribbon file
    inner = str(surface).replace("surf.gii", f"inner.func.gii")
    inner_surf = str(surface).replace("surf.gii", f"inner.surf.gii")
    outer_surf = str(surface).replace("surf.gii", f"outer.surf.gii")
    !wb_command -metric-math x-{offset}*y {inner} -var x {surface_metric} -var y {surface_norm}
    !wb_command -surface-set-coordinates {surface} {inner} {inner_surf}
    !cp {surface} {outer_surf}

    # set structure
    for ix, s in enumerate([inner_surf, outer_surf]):

        if ix == 0:
            secondary_type = "GRAY_WHITE"
        elif ix == 1:
            secondary_type = "PIAL"
        else:
            raise ValueError("`ix` must be 0 or 1.")

        if hemi == 'L':
            structure = "CORTEX_LEFT"
        elif hemi == 'R':
            structure = "CORTEX_RIGHT"
        else:
            raise ValueError("`hemi` must be L or R.")

        !wb_command -set-structure {s} {structure} -surface-type ANATOMICAL -surface-secondary-type {secondary_type}

    # inflate surfaces
    inflated = outer_surf.replace(".outer.", ".inflated.")
    very_inflated = outer_surf.replace(".outer.", ".very_inflated.")
    !wb_command -surface-generate-inflated {outer_surf} {inflated} {very_inflated}
    
    # clean-up
    !rm {surface_metric} {surface_norm} {inner}

parsed 'x-0.5*y' as 'x - (0.5 * y)'
parsed 'x-0.5*y' as 'x - (0.5 * y)'


In [3]:
!ls {mouse_aba_surfaces}

README.txt
sub-043ABA_species-Mus+musculus_hemi-L.inflated.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.inner.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.outer.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.sphere
sub-043ABA_species-Mus+musculus_hemi-L.sphere.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.surf
sub-043ABA_species-Mus+musculus_hemi-L.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L.very_inflated.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere.reg
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere.reg.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.sphere.surf.gii
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.surf
sub-043ABA_species-Mus+musculus_hemi-L_topo-Homo.sapiens.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.inflated.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.inner.surf.gii
sub-043ABA_species-Mus+musculus_hemi-R.outer.sur